In [ ]:
import pandas as pd
import numpy as np
import torch
import torch.nn as nn
import matplotlib.pyplot as plt
import os
from transformers import pipeline, AutoTokenizer

from src.proj_utils import(
    tokenize_text,
    create_training_examples,
    prepare_datasets,
    build_vocabulary
)

from src.model import LSTMLangModel, train_model
from src.dataset import TextGenerationDataset, create_dataloaders
from src.evaluate_transformers import evaluate_distilgpt2_rouge, evaluate_text_generation, compare_models_generation

In [ ]:
df = pd.read_csv('data/training.1600000.processed.noemoticon_reduced.csv', encoding='latin-1', header=None)
df.columns = ['sentiment', 'id', 'date', 'flag', 'user', 'text']

Блок приведения к нижнему регистру, удаления ссылок, упоминаний через @, спецсимоволов.
После - удалить лишние символы

In [ ]:
df['text'] = df['text'].str.lower()
df['text'] = df['text'].str.replace(r'http\S+|www\S+|@\w+|[^\w\s]', '', regex=True)
df['text'] = df['text'].str.strip()

Получение токенов из текста публикаций и обучающих примеров

In [ ]:
df['tokenized'] = df['text'].apply(tokenize_text)

In [ ]:
X, Y = create_training_examples(df['tokenized'].to_list())

Разделение на обучающую проверочную и тестовую выборки

In [ ]:
X_train, X_val, X_test, y_train, y_val, y_test = prepare_datasets(X, Y)

In [ ]:
print(f"Размеры данных:")
print(f"Train: {len(X_train)} примеров")
print(f"Validation: {len(X_val)} примеров")
print(f"Test: {len(X_test)} примеров")

Создать словарь

In [ ]:
all_tokens = []
for seq in X:
    all_tokens.extend(seq)
all_tokens.extend(Y)
vocab = build_vocabulary([all_tokens], min_freq=1)
vocab_size = len(vocab)

print(f"Размер словаря: {vocab_size}")
print(f"Примеры слов в словаре: {list(vocab.keys())[:20]}")


test_words = ["this", "is", "a"]
for word in test_words:
    if word in vocab:
        print(f"'{word}' есть в словаре: индекс {vocab[word]}")
    else:
        print(f"'{word}' НЕТ в словаре!")

Создать загрузчиков данных

In [ ]:
batch_size = 64
max_seq_length = 50

train_loader, val_loader, test_loader = create_dataloaders(
        X_train, y_train, X_val, y_val, X_test, y_test,
        vocab, batch_size, max_seq_length
    )

Создать модель

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Используемое устройство: {device}")

model = LSTMLangModel(
    vocab_size=vocab_size,
    embedding_dim=256,
    hidden_dim=512,
    num_layers=2,
    dropout=0.3
).to(device)
print(f"Количество параметров модели: {sum(p.numel() for p in model.parameters()):,}")

Обучение модели

In [ ]:
train_losses, val_losses, val_rouge_scores = train_model(
        model, train_loader, val_loader, vocab_size, device,
        num_epochs=10, learning_rate=0.001
    )

Визуализация результатов

In [ ]:
plt.figure(figsize=(15,5))

#Потери
plt.subplot(1, 2, 1)
plt.plot(train_losses, label='Train Loss')
plt.plot(val_losses, label='Val Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.title('Training and Validation Loss')
plt.legend()
plt.grid(True)

#Rouge метрики
plt.subplot(1, 2, 2)
rouge1_scores = [score.get('rouge1', 0) for score in val_rouge_scores if score]
rouge2_scores = [score.get('rouge2', 0) for score in val_rouge_scores if score]
rougeL_scores = [score.get('rougeL', 0) for score in val_rouge_scores if score]

epochs = range(1, len(rouge1_scores) + 1)
plt.plot(epochs, rouge1_scores, label='ROUGE-1', marker='o')
plt.plot(epochs, rouge2_scores, label='ROUGE-2', marker='s')
plt.plot(epochs, rougeL_scores, label='ROUGE-L', marker='^')
plt.xlabel('Epoch')
plt.ylabel('ROUGE Score')
plt.title('Validation ROUGE Scores')
plt.legend()
plt.grid(True)

plt.tight_layout()
plt.savefig('training_metrics.png')
plt.show()

print("\nФинальные метрики:")
print(f"Final Train Loss: {train_losses[-1]:.4f}")
print(f"Final Val Loss: {val_losses[-1]:.4f}")
if val_rouge_scores and val_rouge_scores[-1]:
    print(f"Final ROUGE-1: {val_rouge_scores[-1]['rouge1']:.4f}")
    print(f"Final ROUGE-2: {val_rouge_scores[-1]['rouge2']:.4f}")
    print(f"Final ROUGE-L: {val_rouge_scores[-1]['rougeL']:.4f}")

Тестирование модели

In [ ]:
model.eval()
test_loss = 0
criterion = nn.CrossEntropyLoss(ignore_index=0)

with torch.no_grad():
    for batch in test_loader:
        input_ids = batch['input_ids'].to(device)
        targets = batch['target'].to(device)
        
        outputs, _ = model(input_ids)
        loss = criterion(outputs, targets)
        
        test_loss += loss.item()

avg_test_loss = test_loss / len(test_loader)
print(f"Значение потери на этапе тестирования: {avg_test_loss:.4f}")

Генерация связности

In [ ]:
test_cases = [
    (["i", "feel", "so"], "happy today because the sun is shining"),
    (["the", "movie", "was"], "really amazing with great special effects"),
    (["machine", "learning", "is"], "a fascinating field that continues to evolve"),
    (["the", "cat", "is"], "on the map")
]
evaluate_text_generation(model, test_cases, vocab, device)

Оценка с использованием distilgpt2

In [ ]:
val_texts = []
for i, (x_seq, y_token) in enumerate(zip(X_val, y_val)):
    if i < 100:  # Ограничение количества для скорости
        text = ' '.join(x_seq) + ' ' + y_token
        val_texts.append(text)

gpt2_results = evaluate_distilgpt2_rouge(val_texts[:50])
print("\nСравнение моделей:")
print(f"LSTM ROUGE-1: {val_rouge_scores[-1]['rouge1']:.4f}")
print(f"DistilGPT2 ROUGE-1: {gpt2_results['rouge1']:.4f}")


Прямое сравнение генераций LSTM и DistilGPT2 на тестовой выборке

In [ ]:
gpt2_generator = pipeline(
    "text-generation", 
    model="distilgpt2",
    device=device if torch.cuda.is_available() else -1,
    torch_dtype=torch.float16 if torch.cuda.is_available() else torch.float32
)
gpt2_tokenizer = AutoTokenizer.from_pretrained("distilgpt2")
gpt2_tokenizer.pad_token = gpt2_tokenizer.eos_token

# Тестовые примеры из тестовой выборки (возьмите реальные примеры)
test_examples_from_dataset = [
    (X_test[i][-3:], y_test[i]) for i in range(min(10, len(X_test)))
]

print("Сравнение на реальных примерах из тестовой выборки:")
compare_models_generation(
    model, 
    test_examples_from_dataset, 
    vocab, 
    device, 
    gpt2_generator, 
    gpt2_tokenizer
)

Сохранение модели

In [ ]:
save_path = "models/lstm_language_model.pth"
os.makedirs(os.path.dirname(save_path), exist_ok=True)
torch.save({
        'model_state_dict': model.state_dict(),
        'vocab': vocab,
        'model_config': {
            'vocab_size': vocab_size,
            'embedding_dim': 256,
            'hidden_dim': 512,
            'num_layers': 2,
            'dropout': 0.3
        }
    }, save_path)
print(f"Модель сохранена в: {save_path}")

# Вывод

Обучение проходило на выборке в 100 000 записей. Результат на основе LSTM превосходит DistilGPT2:

LSTM ROUGE-1: <b>0.1184</b><br>
DistilGPT2 ROUGE-1: <b>0.0271</b>

Разные сесси обучения на одних и тех же исходных данных не показывают стабильного результата для LSTM по методике ROUGE (0.11 - 0.45))<br/>
Результаты сравнения генерации LSTM и distil-gpt2 представлены выше
